# AuxTel ISR in notebook
Craig Lage 03-Mar-23

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from lsst.daf.butler import Butler
from lsst.summit.utils import BestEffortIsr
from lsst.pipe.tasks.quickFrameMeasurement import QuickFrameMeasurementTask
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask
%matplotlib inline

In [ ]:
def plotExp(exp):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = 0#np.nanpercentile(data, 1)
    vmax = 20#np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure

def repairCosmics(postIsr):
    # Borrowed from summit utils
    if postIsr.getPsf() is None:
        installPsfTask = InstallGaussianPsfTask()
        installPsfTask.run(postIsr)

    # TODO: try adding a reasonably wide Gaussian as a temp PSF
    # and then just running repairTask on its own without any
    # imChar. It should work, and be faster.
    repairConfig = CharacterizeImageTask.ConfigClass()
    repairConfig.doMeasurePsf = False
    repairConfig.doApCorr = False
    repairConfig.doDeblend = False
    repairConfig.doWrite = False
    repairConfig.repair.cosmicray.nCrPixelMax = 200000
    repairTask = CharacterizeImageTask(config=repairConfig)
    repairTask.repair.run(postIsr)
    return

In [ ]:
expId = 2023013100536
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib"])

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doApplyGains=False
isrConfig.doFlat=False
isrConfig.doDefect=True
isrConfig.doNanMasking=True
isrConfig.doInterpolate=True
isrConfig.doSaturation=True
isrConfig.doSaturationInterpolation=True
isrConfig.doWrite=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
%matplotlib
exp = butler.get('raw', detector=0, exposure=expId)
biasExp = butler.get('bias', detector=0, exposure=expId)
#flatExp = butler.get('flat', detector=0, exposure=expId)
#darkExp = butler.get('dark', detector=0, exposure=expId)
defectExp = butler.get('defects', detector=0, exposure=expId)
isrResult = isrTask.run(exp, bias=biasExp, defects=defectExp)
postIsrExp = isrResult.exposure
repairCosmics(postIsrExp)
fig = plotExp(postIsrExp)
plt.savefig("/home/c/cslage/u/AuxTel/crosstalk/Image_2023013100536.png")